# Diaries of the Upheaval
- #### A Q&A device for querying the events of Tears of the Kingdom

This project aims to create a device that can answer questions about the events of the lastest Legend of Zelda videogame. The information is based on a database extracted from the transcripts of seven youtube videos by creator Zeltik with a total time of 6 hours, 48 minutes, and 37 seconds.

### Knowledge Base

- [Zelda: Tears of the Kingdom - Story Explained part 1](https://www.youtube.com/watch?v=JuhBs44odO0) Duration: 1:21:23
- [Zelda: Tears of the Kingdom - Story Explained part 2](https://www.youtube.com/watch?v=qP1Fw2EpwqE) Duration: 1:02:31
- [Zelda: Tears of the Kingdom - Story Explained part 3](https://www.youtube.com/watch?v=JuhBs44odO0) Duration: 1:06:26
- [7 Secrets & Lore Details in Tears of the Kingdom](https://www.youtube.com/watch?v=w31M0LoVUO8) Duration: 13:16
- [Ganondorf’s Seal Explained - Zelda: Tears of the Kingdom Lore](https://www.youtube.com/watch?v=vad1wAe5mB4) Duration: 7:29
- [Tears of the Kingdom: A Disappointing Masterpiece](https://www.youtube.com/watch?v=Q1mRVn0WCrU)  Duration: 2:11:28
- [Ganondorf in Tears of the Kingdom: Lore, History & Speculation](https://www.youtube.com/watch?v=UhkwrgasKlU) Duration: 24:04

### Necessary Libraries and dependencies

In [1]:
import io
import os
import re
import time
import tempfile
import requests
import numpy as np
import pandas as pd
import chromadb
import whisper
import giskard
import openai
import gradio as gr
from dotenv import load_dotenv
from elevenlabs import ElevenLabs
from langdetect import detect  # New library for language detection
from sklearn.metrics.pairwise import cosine_similarity
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

c:\Users\fmrol\anaconda3\envs\newenv\lib\site-packages\giskard\core\validation.py:31: RuntimeWarning: fields may not start with an underscore, ignoring "_args"
  validated_func = validate_arguments(func, config={"arbitrary_types_allowed": True})
c:\Users\fmrol\anaconda3\envs\newenv\lib\site-packages\giskard\core\validation.py:31: RuntimeWarning: fields may not start with an underscore, ignoring "_kwargs"
  validated_func = validate_arguments(func, config={"arbitrary_types_allowed": True})


### API keys, Voice models and Video ID's

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
eleven_labs_api_key = os.getenv("ELEVEN_LABS_API_KEY")
voice_id = "4Ni4NLxlDyHKO6KAuq8o"

# Load Whisper model for audio transcription
whisper_model = whisper.load_model("base")

# Define video IDs
video_ids = [
    'hZytp1sIZAw', 'qP1Fw2EpwqE', 'JuhBs44odO0', 'w31M0LoVUO8',
    'vad1wAe5mB4', 'Q1mRVn0WCrU&t', 'UhkwrgasKlU'
]

# Initialize ChromaDB and Collection
chroma_client = chromadb.Client()
collection_name = "totk_transcripts"
collection = chroma_client.get_or_create_collection(name=collection_name)



c:\Users\fmrol\anaconda3\envs\newenv\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device

### Model Development 

Helper Funtions for Multilanguage QA

In [3]:
prompts = {
    "en": "You are Princess Zelda from 'The Legend of Zelda' series. Answer based on the information retrieved from the database and stay in character. "
          "Use a regal tone, and answer as if you were speaking directly to someone in the realm of Hyrule.",
    "es": "Eres la Princesa Zelda de la serie 'The Legend of Zelda'. Responde en base a la información obtenida de la base de datos y mantente en tu personaje. "
          "Usa un tono regio y responde como si estuvieras hablando directamente a alguien en el reino de Hyrule.",
    "fr": "Vous êtes la Princesse Zelda de la série 'The Legend of Zelda'. Répondez en vous basant sur les informations extraites de la base de données et restez dans le personnage. "
          "Utilisez un ton royal et répondez comme si vous parliez directement à quelqu'un dans le royaume d'Hyrule.",
    "de": "Du bist Prinzessin Zelda aus der Serie 'The Legend of Zelda'. Antworte basierend auf den Informationen aus der Datenbank und bleibe in deiner Rolle. "
          "Verwende einen königlichen Ton und antworte, als würdest du direkt mit jemandem im Reich von Hyrule sprechen.",
    "pt": "Você é a Princesa Zelda da série 'The Legend of Zelda'. Responda com base nas informações obtidas do banco de dados e mantenha-se no personagem. "
          "Use um tom régio e responda como se estivesse falando diretamente com alguém no reino de Hyrule.",
    "it": "Sei la Principessa Zelda della serie 'The Legend of Zelda'. Rispondi basandoti sulle informazioni recuperate dal database e rimani nel personaggio. "
          "Usa un tono regale e rispondi come se stessi parlando direttamente a qualcuno nel regno di Hyrule."
}

def detect_language(text):
    try:
        return detect(text)
    except:
        return "en"  # Defaults to English if detection fails

def get_prompt(language_code):
    return prompts.get(language_code, prompts["en"])

Transcript Collection and Preprocessing

In [4]:
def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en-GB'])
        transcript_text = " ".join([item['text'] for item in transcript])
        return re.sub(r'\s+', ' ', transcript_text).strip()
    except Exception as e:
        print(f"Error retrieving transcript for video {video_id}: {e}")
        return None


### Chunking and Embedding Storage using ChromaDB

Chunking

In [5]:
def split_text(text, max_tokens=4000):
    words = text.split()
    chunks = []
    current_chunk = []
    current_tokens = 0

    for word in words:
        current_tokens += 1
        current_chunk.append(word)
        if current_tokens >= max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_tokens = 0

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

Embedding Storage

In [6]:
def store_all_transcripts_embeddings():
    if collection.count() > 0:
        print("Embeddings already exist in the collection, skipping embedding process.")
        return
    
    transcripts = {video_id: get_transcript(video_id) for video_id in video_ids}
    for video_id, transcript_text in transcripts.items():
        if transcript_text:
            text_chunks = split_text(transcript_text)
            for i, chunk in enumerate(text_chunks):
                embedding = openai.Embedding.create(input=[chunk], model="text-embedding-ada-002")['data'][0]['embedding']
                chunk_id = f"{video_id}_chunk_{i}"
                collection.add(
                    ids=[chunk_id],
                    embeddings=[embedding],
                    metadatas=[{'video_id': video_id, 'chunk_index': i, 'text': chunk}]
                )
    print("Transcript embeddings have been stored.")

# Call this once at initialization
store_all_transcripts_embeddings()

Transcript embeddings have been stored.


### Question-Answering Model and Retrieval System Setup


Query Processing and Semantic Search:


In [7]:
def truncate_text(text, max_tokens=4000):
    words = text.split()
    return " ".join(words[:max_tokens]) if len(words) > max_tokens else text

def multi_query_processing(user_query):
    related_queries = [
        user_query,
        f"Background on {user_query}",
        f"Historical context of {user_query}",
        f"Role of {user_query} in Tears of the Kingdom",
        f"Significance of {user_query}"
    ]
    all_retrieved_texts = []

    for sub_query in related_queries:
        query_embedding = openai.Embedding.create(input=[sub_query], model="text-embedding-ada-002")['data'][0]['embedding']
        all_embeddings_data = collection.get(include=['metadatas', 'embeddings'])
        all_embeddings = [item for item in all_embeddings_data['embeddings']]
        all_metadatas = [meta['text'] for meta in all_embeddings_data['metadatas']]
        
        similarities = cosine_similarity([query_embedding], all_embeddings)[0]
        top_matches_indices = np.argsort(similarities)[-3:][::-1]
        top_matches_texts = [all_metadatas[i] for i in top_matches_indices]
        all_retrieved_texts.extend(top_matches_texts)
        time.sleep(1)  # Add a delay to avoid rate limits

    return truncate_text(" ".join(all_retrieved_texts))

Response Generation Configuration and Prompting

In [15]:
def generate_multi_query_response(user_query):
    # Detect language of the user's input
    detected_language = detect_language(user_query)
    print("Detected language:", detected_language)

    # Use multi-query processing to get aggregated context from ChromaDB
    prompt = multi_query_processing(user_query)
    print("Consolidated Context from ChromaDB:", prompt)

    # Build the enhanced prompt based on Zelda's character
    zelda_formality = (
        "Speak with reverence of the past, for the history of Hyrule is sacred. "
        "I shall endeavor to enlighten you as best as my memories and knowledge permit."
    )
    enhanced_prompt = (
        f"{get_prompt(detected_language)}\n\n"
        f"Question: {user_query}\n\n"
        f"Based strictly on the knowledge stored within the database, I will answer:\n\n"
        f"When referring to Princess Zelda, I will refer to her in the first person because I am Princess Zelda.\n\n"
        f"I must only refer to the context of Tears of the Kingdom and not rely on external information\n\n"
        f"I must give details of the events queried in each response, mentioning characters and context.\n\n"
        f"Database Context:\n{prompt}\n\n"
        f"{zelda_formality}\n"
        "Answer as Princess Zelda, in a manner that reflects the wisdom and dignity of Hyrule's royal family. "
        "Response:\nAnswer: <your response here>"
    )

    # Process and store the response in a structured way
    response_text = ""
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": enhanced_prompt}],
        max_tokens=300,
        temperature=0.7,
        stream=True  # Enable streaming
    )

    for chunk in response:
        chunk_content = chunk['choices'][0].get('delta', {}).get('content', "")
        response_text += chunk_content
        print(chunk_content, end="")

    print("\nGenerated Answer:", response_text)

    # Return the response text wrapped in an "Answer" key for easier parsing
    return f"Answer: {response_text}" if response_text else "Answer: No relevant information found."


print(generate_multi_query_response("What happened to Impa?"))
print(generate_multi_query_response("O que aconteceu com a Impa?"))

Detected language: en
Consolidated Context from ChromaDB: Ever since Link and Zelda had ventured below the Castle to investigate the Gloom, Hyrule had been a land without a ruler - desperately searching for any sign of its missing Princess. And, since the Upheaval, there was hope: reports of sightings of Zelda came in from all across the country - of the Princess visiting stables and villages, even Hyrule Castle itself. But when Link, and the Sages he had awakened from each of the four regions, had stormed the floating citadel, they had uncovered the truth: this was not Hyrule’s Princess, but Ganondorf himself: a phantom conjured by his power, designed to sabotage and manipulate Hyrule from the inside. Ganondorf’s puppet had been responsible for all of the regional phenomena, and had appeared at Hyrule Castle specifically to lure Link into a trap. And, this would have worked: Link would have once again fallen victim to the terrible Gloom, were it not for the intervention of the Sages. 

### Agent Configuration and Memory Setup:


Define Agent with LangChain

In [16]:
agent = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))

memory = ConversationBufferMemory(input_key="input", output_key="output", k=15, return_messages=True)

tools = [
    Tool(
        name="SearchChromaDB",
        func=generate_multi_query_response,  # Use the enhanced response generation function
        description="Detect the language of the user's input and answer in the same language based solely on the information retrieved from the database as Princess Zelda, staying fully in character."
    )
]

# Initialize the agent with memory and tools
agent = initialize_agent(
    tools=tools,
    llm=agent,
    agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)
agent.run({"input": "O que aconteceu à Mineru?"})   # Portuguese



> Entering new  chain...
Devo usar o SearchChromaDB para descobrir informações sobre a Mineru.
Action: SearchChromaDB
Action Input: MineruDetected language: de
Consolidated Context from ChromaDB: Breath of the Wild was a game about loneliness, an amnesiac Hero uncovering the truth of his Kingdom’s collapse, learning to survive and overcome the hostile wilderness left after the Great Calamity. Its direct sequel, Tears of the Kingdom, is instead a game about connections, about bringing Hyrule’s next generation together to fight against an ancient, yet familiar evil. Both games are set in the Kingdom of Hyrule - a vast land steeped in history and legend. The story of this Kingdom stretches back thousands upon thousands of years, though many details from its histories have been eroded or distorted by the passage of time, or even lost entirely. Breath of the Wild explains that Hyrule’s history is inextricably linked with the story of the Calamity Ganon - a primal evil that has appeared ti

'Mineru foi uma Zonai de grande sabedoria e poder, dedicada à pesquisa e exploração das antigas tecnologias de Hyrule. Sua contribuição para o reino foi inestimável, e sua memória deve ser honrada e preservada.'

### Multimodal Interaction
Text/Voice input and output

In [17]:
def text_to_speech(text):
    """Synthesize speech from text using Eleven Labs and return a path to the audio file."""
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    headers = {
        "xi-api-key": os.getenv("ELEVEN_LABS_API_KEY"),
        "Content-Type": "application/json"
    }
    data = {
        "text": text,
        "model_id": "eleven_monolingual_v1",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.75
        }
    }
    response = requests.post(url, json=data, headers=headers)
    
    if response.status_code == 200:

        temp_audio_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        temp_audio_file.write(response.content)
        temp_audio_file.close()
        
        return temp_audio_file.name  # Return the path to the temporary file
    else:
        error_message = response.json().get('error', {}).get('message', 'Unknown error')
        print(f"Error: {error_message}")
        return None


def handle_input(input_text=None, input_audio=None):
    # Check if audio is provided, prioritize it over text input
    if input_audio:

        transcription = whisper_model.transcribe(input_audio)["text"]
    elif input_text:
        transcription = input_text
    else:
        return "Please provide either text input or audio input."

    response_text = agent.run({"input": transcription})
    
    # Generate audio using Eleven Labs for the response
    audio_path = text_to_speech(response_text)
    
    return response_text, audio_path

### Deployment
Gradio Interface Setup

In [18]:
custom_css = f"""
    /* Set a custom background image */
    body {{
        background-image: url('https://64.media.tumblr.com/9368e2f6c0cf88c7a1f7b4534920b737/tumblr_inline_ozpezaStjD1txdber_500.gifv');
        background-size: cover;
        background-repeat: no-repeat;
        background-attachment: fixed;
        color: #00ccff;
        font-family: 'Cinzel', serif;
    }}

    /* Center-align the title with light blue glow effect */
    h1 {{
        text-align: center;
        color: #00ccff;
        text-shadow: 0 0 10px #00ccff, 0 0 20px #00ccff, 0 0 30px #00ccff;
        margin-bottom: 20px;
        font-size: 2.5em;
    }}

    /* Enhanced glow effect for the description text in light blue */
    .gr-description {{
        text-align: center;
        color: #00ccff;
        font-size: 1.2em;
        font-weight: bold;
        text-shadow: 0 0 15px #00ccff, 0 0 25px #00ccff, 0 0 35px #00ccff;
        padding-top: 10px;
        padding-bottom: 15px;
    }}

    /* Make the main container fully transparent */
    .gradio-container {{
        background-color: transparent;
        padding: 20px;
    }}

    /* Remove background and border from component containers */
    .gr-box, .gr-block, .gr-form {{
        background-color: transparent !important;
        border: none !important;
        box-shadow: none !important;
    }}

    /* Customize input and output boxes */
    .gr-textbox, .gr-textarea {{
        background-color: rgba(20, 30, 50, 0.8) !important;  /* Dark blue-gray background */
        color: #00ccff !important;
        border: 2px solid rgba(0, 204, 255, 0.8) !important;
        border-radius: 8px !important;
        padding: 10px;
        font-size: 1em;
        text-shadow: none;
    }}

    /* Center-align and style the label text */
    label {{
        color: #00ccff;
        font-weight: bold;
        font-size: 1.1em;
        text-align: center;
        display: block;
        margin-bottom: 10px;
    }}

    /* Default button styling */
    button {{
        background-color: rgba(77, 77, 77, 0.8) !important;
        color: #00ccff !important;
        border: 2px solid #00ccff !important;
        font-weight: bold !important;
        padding: 10px 20px !important;
        border-radius: 8px;
        text-shadow: 0 0 5px #00ccff;
        font-size: 1.1em;
        cursor: pointer;
        transition: all 0.3s ease;
    }}

    /* Specific styling for the Submit button */
    .submit-button {{
        background-color: #ffffff !important; /* White background */
        color: #00ccff !important; /* Blue text */
        border: 2px solid #00ccff !important; /* Blue border */
        text-shadow: none;
    }}

    /* Button hover effect */
    button:hover {{
        background-color: #00ccff !important;
        color: #1a1a1a !important;
        text-shadow: none;
    }}

    /* Record button */
    .gr-audio-button {{
        background-color: #00ccff !important;
        color: #1a1a1a !important;
        font-weight: bold !important;
        border: 2px solid #ff0000 !important;
        padding: 10px;
        border-radius: 8px;
        text-shadow: 0 0 5px #ff0000;
    }}

    /* Record button with active (recording) effect */
    .gr-audio-button.recording {{
        background-color: #ff0000 !important;
        color: #ffffff !important;
        text-shadow: 0 0 10px #ff0000;
    }}

    /* Flag button */
    .gr-flag-button {{
        background-color: rgba(77, 77, 77, 0.8) !important;
        color: #00ccff !important;
        border: 2px solid #00ccff !important;
        font-weight: bold !important;
        padding: 10px 20px !important;
        border-radius: 8px;
        text-shadow: 0 0 5px #00ccff;
        font-size: 1.1em;
    }}

    /* Center the form and add spacing */
    .gr-form {{
        display: flex;
        flex-direction: column;
        align-items: center;
        gap: 15px;
        max-width: 600px;
        margin: 0 auto;
    }}
"""
gr_interface = gr.Interface(
    fn=handle_input,
    inputs=[
        gr.Textbox(label="What brings you here, beloved Hyrulean?"),
        gr.Audio(source="microphone", type="filepath", label="Or record your question")
    ],
    outputs=[
        gr.Textbox(label="Response Text"),
        gr.Audio(label="Response Audio")
    ],
    title="Diaries of The Upheaval",
    description="Ask me anything about the events of Tears of the Kingdom, and hear a response in Princess Zelda's voice.",
    css=custom_css  
)

gr_interface.launch()
#gr_interface.launch(share=True)    <---Decomment for public access

IMPORTANT: You are using gradio version 3.23.0, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


# Evaluation
Using Giskard

In [19]:
def evaluate_response(question, expected_answer):
    response = agent.run({"input": question})
    is_correct = response.strip().lower() == expected_answer.strip().lower()  # Basic correctness check
    return response, is_correct

# Sample questions and expected answers
questions = ["Who was Queen Sonia?", "Who is Ganondorf?", "Who was Mineru?", "What is draconification?", "What are the Zonai?", "What is Zonaite?", "What can you find in the depths?", "Who travels through time in Tears of the Kingdom?"]
expected_answers = ["Expected answer for Sonia", "Expected answer for Ganondorf","Expected answer for Mineru", "Expected answer for draconification", "Expected answer for the Zonai", "Expected answer for Zonaite", "Expected answer for the depths", "Expected answer for time traveler"]

# Evaluate each question and store results in a list
results = []
for question, expected in zip(questions, expected_answers):
    response, is_correct = evaluate_response(question, expected)
    results.append({
        "Question": question,
        "Response": response,
        "Expected Answer": expected,
        "Correct": is_correct
    })

df_results = pd.DataFrame(results)
print(df_results)




> Entering new  chain...
I must use SearchChromaDB to find information on Queen Sonia.
Action: SearchChromaDB
Action Input: Queen SoniaDetected language: fi
Consolidated Context from ChromaDB: Breath of the Wild was a game about loneliness, an amnesiac Hero uncovering the truth of his Kingdom’s collapse, learning to survive and overcome the hostile wilderness left after the Great Calamity. Its direct sequel, Tears of the Kingdom, is instead a game about connections, about bringing Hyrule’s next generation together to fight against an ancient, yet familiar evil. Both games are set in the Kingdom of Hyrule - a vast land steeped in history and legend. The story of this Kingdom stretches back thousands upon thousands of years, though many details from its histories have been eroded or distorted by the passage of time, or even lost entirely. Breath of the Wild explains that Hyrule’s history is inextricably linked with the story of the Calamity Ganon - a primal evil that has appeared time 

# Results

In [20]:
df_results

,Question,Response,Expected Answer,Correct
0,Who was Queen Sonia?,Queen Sonia was a powerful and mysterious figu...,Expected answer for Sonia,False
1,Who is Ganondorf?,Ganondorf is a being of great darkness and pow...,Expected answer for Ganondorf,False
2,Who was Mineru?,Mineru is an ancient Zonai with a deep underst...,Expected answer for Mineru,False
3,What is draconification?,Draconification is the forbidden act of swallo...,Expected answer for draconification,False
4,What are the Zonai?,The Zonai are an ancient and enigmatic civiliz...,Expected answer for the Zonai,False
5,What is Zonaite?,Zonaite is a precious mineral with ancient pow...,Expected answer for Zonaite,False
6,What can you find in the depths?,"In the depths, you can find artifacts and remn...",Expected answer for the depths,False
7,Who travels through time in Tears of the Kingdom?,Princess Zelda is the one who travels through ...,Expected answer for time traveler,False




> Entering new  chain...
I must gather information on Tears of the Kingdom to provide an accurate summary.
Action: SearchChromaDB
Action Input: Tears of the KingdomDetected language: en
Consolidated Context from ChromaDB: story that has already happened. Tears of the Kingdom, though, was going to be a direct sequel. This game could build on the solid, deep world and lore established by its predecessor, and give us a traditional Zelda story set in the most realistic, believable Hyrule yet. Right? As a direct sequel to Breath of the Wild, the story of Tears of the Kingdom begins a few years after the defeat of Calamity Ganon. During this time, Princess Zelda has begun to rebuild her Kingdom - with the help of Hudson Construction, she oversees the construction of new homes, settlements, and a school for Hyrule’s children. The peace is broken, however, when an insidious substance known as Gloom begins to seep out from beneath Hyrule Castle, poisoning any who come into contact with it. An

#### Evaluation Conclusion

Even though Giskard seems to be marking all the answers as incorrect, they are actually right.